<a href="https://colab.research.google.com/github/alexmjn/DS-Unit-1-Sprint-2-Statistics/blob/master/module3/LS_DS_123_Introduction_to_Bayesian_Inference_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Lambda School Data Science Module 123

## Introduction to Bayesian Inference




## Assignment - Code it up!

We used pure math to apply Bayes Theorem to drug tests. Now write Python code to reproduce the results! This is purposefully open ended - you'll have to think about how you should represent probabilities and events. You can and should look things up.

Specific goals/targets:

### 1) Write a function 

`def prob_drunk_given_positive(prob_drunk_prior, false_positive_rate, true_positive_rate):` 

You should only truly need these three values in order to apply Bayes Theorem. In this example, imagine that individuals are taking a breathalyzer test with an 8% false positive rate, a 100% true positive rate, and that our prior belief about drunk driving in the population is 1/1000. Be able to call the function iteratively through a for loop. 
 - What is the probability that a person is drunk after one positive breathalyzer test?
 - What is the probability that a person is drunk after two positive breathalyzer tests?
 - How many positive breathalyzer tests are needed in order to have a probability that's greater than 95% that a person is drunk beyond the legal limit?

### 2) Explore `scipy.stats.bayes_mvs`  
Read its documentation, and experiment with it on data you've tested in other ways earlier this week.
 - Create a visualization comparing the results of a Bayesian approach to a traditional/frequentist approach. (with a large sample size they should look close to identical, however, take this opportunity to practice visualizing condfidence intervals in general. The following are some potential ways that you could visualize confidence intervals on your graph:
  - [Matplotlib Error Bars](https://matplotlib.org/3.1.1/api/_as_gen/matplotlib.pyplot.errorbar.html)
  - [Seaborn barplot with error bars](https://seaborn.pydata.org/generated/seaborn.barplot.html)
  - [Vertical ines to show bounds of confidence interval](https://www.simplypsychology.org/confidence-interval.jpg)
  - [Confidence Intervals on Box Plots](https://matplotlib.org/3.1.1/api/_as_gen/matplotlib.axes.Axes.boxplot.html)

### 3) In your own words, summarize the difference between Bayesian and Frequentist statistics

If you're unsure where to start, check out [this blog post of Bayes theorem with Python](https://dataconomy.com/2015/02/introduction-to-bayes-theorem-with-python/).



In [0]:
def prob_drunk_given_positive(prob_drunk_prior, false_positive_rate, true_positive_rate):
  """
  Use Bayes' Theorem to calculate the probability that someone who failed a
  breathalyzer is drunk.
  """
  numerator = (prob_drunk_prior * true_positive_rate)
  denominator = ((prob_drunk_prior * true_positive_rate) + ((1 - prob_drunk_prior) * false_positive_rate))
  posterior_probability = numerator/denominator
  return posterior_probability

In [4]:
prob_drunk_given_positive(1/1000, .08, 1)

0.012357884330202669

In [5]:
prob_drunk_given_positive(prob_drunk_given_positive(1/1000, .08, 1), .08, 1)

0.13525210993291495

Under these assumptions, someone who fails a breathalyzer once is only 1.2% to be drunk! After two tests, that increases, but still only to 13.5%.

In [0]:
def prob_dd_iterator(n, prob_drunk_prior = .001,
                     false_positive_rate = .08, true_positive_rate = 1):
  """
  Simulates n breathalyzer tests.
  
  Takes a number of positive tests, the population rate of drunk driving,
  the false positive rate of the test, and the rate at which the test
  detects true drunk driving.
  
  Returns the probability the driver is drunk.
  """
  for i in range(n):
    prob_drunk_prior = prob_drunk_given_positive(prob_drunk_prior,
                                                  false_positive_rate,
                                                  true_positive_rate)
  return prob_drunk_prior  

In [7]:
# I verify this produces the desired output 
prob_dd_iterator(2)

0.13525210993291495

In [0]:
def calculate_minimum_tests(p = .95, 
                            prob_drunk_prior = .001,
                            false_positive_rate = .08, 
                            true_positive_rate = 1):
  
  """
  Calculates the minimum number of positive breathalyzer tests to guarantee that 
  the probability of the driver actually being drunk is higher than the specified
  threshold.
  """
  tests = 0
  while prob_drunk_prior < p:
    tests += 1
    prob_drunk_prior = prob_drunk_given_positive(prob_drunk_prior,
                                                  false_positive_rate,
                                                  true_positive_rate)
  return tests



In [10]:
calculate_minimum_tests()

4

In [11]:
prob_dd_iterator(4)

0.9606895076105054

The minimum_tests function shows that we need 4 positive tests to have more than a .95 probability that a driver is drunk.

**Frequentist statistics** offer a clean method of concluding whether data is significant or insignificant. The appeal is that it's relatively easy to calculate and interpret* p-values. 

  *: not really, but it's easy to think you know how to. 

**Bayesian statistics** aren't as clean, and require the use of prior probabilities, which may not be as cut-and-dry as frequentist statistics. They use prior probabilities and observed evidence to produce an estimate of the posterior probability. (Over time and repeated applications, both methods will converge.)  

  However, there are several conceptual issues with frequentist statistics. Firstly, we know that the null hypothesis is never true. The true population effect is essentially never *truly* zero in something we would be looking at. So the real question is one of effect size, which the binary interpretations of frequentist statistics obscure.

  Secondly: we rarely truly have zero prior knowledge. In fact, many of the worst replication-crisis offenders would have been impossible with a need to justify a prior belief. For example, experiments that tied voting preferences to "time within menstrual cycle" would run up against a body of literature that strongly suggested electoral choices are very, very stable over time.

  Finally, the ease of implicitly running many different experiments (how do the subgroups look? what about interaction effects?) means that, without being pinned down to realistic prior estimates of effect sizes, it is really, really easy to use frequentist statistics in a way that generates results that are not just overblown but completely uncorrelated with reality. We now know that entire bodies of psychological literature were based on statistical noise. Years and years of work have literally no value.

[What has happened down here, is that the winds have changed.](https://statmodeling.stat.columbia.edu/2016/09/21/what-has-happened-down-here-is-the-winds-have-changed/)

  Now, it's not fair to pin this entirely on frequentist statistics. It can be done well or poorly. The incentive structure in academia, a certain lack of trendiness in theory and methods, and a media willing to uncritically accept claims all contributed. But, at the same time, it's hard to argue things would have gotten so bad without the widespread usage of significance testing and p-values.


## Resources

- [Worked example of Bayes rule calculation](https://en.wikipedia.org/wiki/Bayes'_theorem#Examples) (helpful as it fully breaks out the denominator)
- [Source code for mvsdist in scipy](https://github.com/scipy/scipy/blob/90534919e139d2a81c24bf08341734ff41a3db12/scipy/stats/morestats.py#L139)

## Stretch Goals:

- Go back and study the content from Modules 1 & 2 to make sure that you're really comfortable with them.
- Apply a Bayesian technique to a problem you previously worked (in an assignment or project work) on from a frequentist (standard) perspective
- Check out [PyMC3](https://docs.pymc.io/) (note this goes beyond hypothesis tests into modeling) - read the guides and work through some examples
- Take PyMC3 further - see if you can build something with it!